In [12]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

In [13]:
# CSV-Datei laden (simulierte Daten)
daten = pd.read_csv('Maschinendaten.csv')

# Merkmale (Features) und Zielvariable (Target) festlegen
merkmale = daten[['Betriebszeit', 'Wartungskosten', 'Maschinenleistung']]  # Merkmale
zielvariable = daten['Energieverbrauch']  # Zielvariable

# Daten in Trainings- und Testdaten aufteilen
merkmale_train, merkmale_test, ziel_train, ziel_test = train_test_split(merkmale, zielvariable, test_size=0.2, random_state=42)

In [14]:
# Lineare und Multilineare Regression für Vergleich:

# 2. Lineare Regression (einfach) - nur ein Merkmal
lineares_modell = LinearRegression()
lineares_modell.fit(merkmale_train[['Betriebszeit']], ziel_train)
vorhersage_linear = lineares_modell.predict(merkmale_test[['Betriebszeit']])

# Berechnung des R²-Wertes für die lineare Regression
r2_linear = r2_score(ziel_test, vorhersage_linear)
print(f"R² der linearen Regression: {r2_linear}")

# 3. Multilineare Regression - alle Merkmale
multilineares_modell = LinearRegression()
multilineares_modell.fit(merkmale_train, ziel_train)
vorhersage_multilinear = multilineares_modell.predict(merkmale_test)

# Berechnung des R²-Wertes für die multiliniere Regression
r2_multilinear = r2_score(ziel_test, vorhersage_multilinear)
print(f"R² der multilinearen Regression: {r2_multilinear}")

R² der linearen Regression: 0.004418875322485638
R² der multilinearen Regression: 0.9659077613893823


In [22]:
# 1. Polynomiale Regression (Nichtlinear) mit Betriebszeit und Wartungskosten
polynom = PolynomialFeatures(degree=2)
merkmale_train_poly = polynom.fit_transform(merkmale_train[['Betriebszeit', 'Wartungskosten']])
merkmale_test_poly = polynom.transform(merkmale_test[['Betriebszeit', 'Wartungskosten']])
polynomiales_modell = LinearRegression()
polynomiales_modell.fit(merkmale_train_poly, ziel_train)
vorhersage_poly = polynomiales_modell.predict(merkmale_test_poly)

# Berechnung der R²-Werte und Fehler
r2_poly = r2_score(ziel_test, vorhersage_poly)
fehler_poly = mean_squared_error(ziel_test, vorhersage_poly)
print(f"R² der polynomialen Regression: {r2_poly}")
print(f"Mean Squared Error der polynomialen Regression: {fehler_poly}")

# Parameter des Modells
koeffizienten = polynomiales_modell.coef_  # Koeffizienten der Polynomial-Features
intercept = polynomiales_modell.intercept_  # Achsenabschnitt

feature_namen = polynom.get_feature_names_out(['Betriebszeit', 'Wartungskosten'])
for name, koeff in zip(feature_namen, koeffizienten):
    print(f"{name}: {koeff}")

R² der polynomialen Regression: 0.9673281638806899
Mean Squared Error der polynomialen Regression: 57738.324157072384
Koeffizienten:
1: 0.0
Betriebszeit: 0.6720654236828563
Wartungskosten: 2.83521926344626
Betriebszeit^2: 4.003306305656296e-05
Betriebszeit Wartungskosten: -0.0004263830509878379
Wartungskosten^2: 0.008359968212861658


In [17]:
# 3D-Plot: Streudiagramm der echten Daten
fig_3d_poly = px.scatter_3d(x=merkmale_test['Betriebszeit'], 
                            y=merkmale_test['Wartungskosten'], 
                            z=ziel_test, 
                            labels={'x': 'Betriebszeit', 'y': 'Wartungskosten', 'z': 'Energieverbrauch'},
                            title='Polynomiale Regression: Energieverbrauch basierend auf Betriebszeit und Maschinenleistung')

# 3. Erstellen einer Regressionsebene
betriebszeit_range = np.linspace(merkmale_test['Betriebszeit'].min(), merkmale_test['Betriebszeit'].max(), 20)
wartungskosten_range = np.linspace(merkmale_test['Wartungskosten'].min(), merkmale_test['Wartungskosten'].max(), 20)

# Gitter erstellen
betriebszeit_grid, wartungskosten_grid = np.meshgrid(betriebszeit_range, wartungskosten_range)
gitter_werte = np.c_[betriebszeit_grid.ravel(), wartungskosten_grid.ravel()]
gitter_werte_poly = polynom.transform(gitter_werte)

# Vorhersagen für die Gitterpunkte
energieverbrauch_grid = polynomiales_modell.predict(gitter_werte_poly).reshape(betriebszeit_grid.shape)

# Regressionsebene zum Plot hinzufügen
fig_3d_poly.add_trace(go.Surface(x=betriebszeit_grid, 
                                 y=wartungskosten_grid, 
                                 z=energieverbrauch_grid, 
                                 name='Regressionsebene', 
                                 colorscale='Viridis', 
                                 opacity=0.5))

# Plot anzeigen
fig_3d_poly.show()

D:\Lehre\Zeitreihen_Analyse_und_Prädiktion\venv\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but PolynomialFeatures was fitted with feature names

